In [1]:
import pandas as pd
sample_submission = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
sample_submission

,id,label
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5
...,...,...
12495,12496,0.5
12496,12497,0.5
12497,12498,0.5
12498,12499,0.5


In [2]:
import zipfile
path_to_zip_file = '/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip'
with zipfile.ZipFile(path_to_zip_file,'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')


In [3]:
path_to_zip_file = '/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip'
with zipfile.ZipFile(path_to_zip_file,'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

In [4]:
import os 
from PIL import Image

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
import numpy as np 
import pandas as pd 
from PIL import Image
from matplotlib import pyplot as plt

In [6]:
a = os.listdir('/kaggle/working/train')
img1 = Image.open(os.path.join('/kaggle/working/train',a[0]))
print(img1.size)

(226, 240)


In [7]:
b=[]
for line in os.listdir('/kaggle/working/train'):
    b.append(line.split('.')[0])
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(b)


LabelEncoder()

In [8]:
class CustomDataset(Dataset):
    def __init__(self,directorie,transform):
        label1=[]
        filename_image=[]
        self.directory = directorie
        for line in os.listdir(directorie):
            k = line
            k = k.split('.')
            label1.append(k[0])
            filename_image.append(line)
        self.label = label1
        self.filename = filename_image
        self.transform = transform
    def __len__(self):
        return len(self.label)
    def __getitem__(self,idx):
        img = Image.open(os.path.join(self.directory,self.filename[idx]))
        if self.transform:
            img = self.transform(img)
        val = le.transform([self.label[idx]])[0]
        return img,val

In [9]:
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])
train_dataset = CustomDataset('/kaggle/working/train',transform=transform)
train_dataloader = DataLoader(train_dataset,batch_size = 32,shuffle = True)

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3,20,3,padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.MaxPool2d(2)
        )
        self.flatten = nn.Flatten()
        self.edges1 = nn.Linear(16*16*20,64)
        self.edges2 = nn.Linear(64,2)
        self.activation = nn.ReLU()

    def forward(self,x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.edges1(x)
        x = self.activation(x)
        x = self.edges2(x)
        return x

In [11]:
device = 'cuda'

In [12]:
model = CNN().to(device)

cr = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)
epochs = 10

corrects = 0
for i in range(epochs):
    print(i)
    cost = 0
    for image,answer in train_dataloader:
        image, answer = image.to(device), answer.to(device)
        optimizer.zero_grad()
        predictions = model(image)
        error = cr(predictions,answer)
        error.backward()
        optimizer.step()
        cost += error.item()
        if i == epochs - 1:
            for j in range(len(answer)):
                if torch.argmax(predictions[j]).item() == answer[j].item():
                    corrects += 1


0
1
2
3
4
5
6
7
8
9


In [13]:
sample_submission = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')

In [14]:
class CustomDataset1(Dataset):
    def __init__(self,directorie,transform):
        filename_image=[]
        self.directory = directorie
        for line in os.listdir(directorie):
            filename_image.append(line)
        self.filename = filename_image
        self.transform = transform
    def __len__(self):
        return len(self.filename)
    def __getitem__(self,idx):
        img = Image.open(os.path.join(self.directory,self.filename[idx]))
        if self.transform:
            img = self.transform(img)
        f = self.filename[idx]
        f = str(f)
        f = f[0:len(f)-4]
        return img,f

In [15]:
test_dataset = CustomDataset1('/kaggle/working/test',transform=transform)
test_dataloader = DataLoader(test_dataset,batch_size = 32,shuffle = False)

In [16]:
for image,filename in test_dataloader:
    image, filename = image.to(device), filename
    optimizer.zero_grad()
    predictions = model(image)
    for j in range(len(filename)):
        sample_submission.loc[int(filename[j])-1,'label'] = torch.softmax(predictions[j], dim=0)[1].item()

In [17]:
sample_submission.to_csv('submission.csv',index=False)

In [18]:
sample_submission

,id,label
0,1,0.533714
1,2,0.670813
2,3,0.669964
3,4,0.881864
4,5,0.051437
...,...,...
12495,12496,0.009046
12496,12497,0.015429
12497,12498,0.658111
12498,12499,0.749565
